# Retraining an existing model


In [ ]:
import time
import json
from pprint import pprint
import boto3

In [ ]:
ENV = 'qa'

In [ ]:
RETRAIN_RECIPES = ['arn:aws:personalize:::recipe/aws-personalized-ranking',
                   'arn:aws:personalize:::recipe/aws-similar-items']

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Gather Dataset and Solution Arn's

List existing datasets.  Datasets have been created by cloudforamtion.

In [ ]:
dataset_groups = personalize.list_dataset_groups()

The following step will now identify the correct dataset group based on the `ENV` suffix

In [ ]:
for group in dataset_groups['datasetGroups']:
    if group['name'].split('-')[0] == ENV:
        dataset_group_arn = (group['datasetGroupArn'])
        
print(dataset_group_arn)

In [ ]:
solution_list = []
solutions = personalize.list_solutions(datasetGroupArn = dataset_group_arn)
# pprint(solutions)

for solution in solutions['solutions']:
    if solution['name'].split('-')[0] == ENV and solution['recipeArn']  in RETRAIN_RECIPES:
        solution_list.append(solution)

pprint(solution_list)
        

## Create New Versions

In [ ]:
version_list = []

for solution in solution_list:
    solution_arn = solution['solutionArn']
    solution_name = solution['name']
    try:
        print(f"Creating solution version {solution_arn}")
        response = personalize.create_solution_version(solutionArn = solution_arn)
        version_list.append({'solution_version_arn': response['solutionVersionArn'],
                            'solution_name': solution_name})
        print(json.dumps(response, indent=2))
    except Exception as e:
        print(f'Solution version {solution_name} failed')
        print(e)

In [ ]:
version_arn_list = [x['solution_version_arn'] for x in version_list]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for solution_version_arn in version_arn_list:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        print(f"Status for {solution_version_arn}: {status}")
        
        if status == "ACTIVE":
            print(f"Build succeeded for {solution_version_arn}")
            version_arn_list.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print(f"Build failed for {solution_version_arn}")
            version_arn_list.remove(solution_version_arn)
    
    if len(version_arn_list) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

## Repoint Campaigns

In [ ]:
campaign_list = personalize.list_campaigns()['campaigns']
pprint(campaign_list)

In [ ]:
for version in version_list:
    solution_version_arn = version['solution_version_arn']
    solution_name = version['solution_name']
    for campaign in campaign_list:
        if campaign['name'] == solution_name:
            print(solution_name, campaign['campaignArn'], solution_version_arn)
            try:
                response = personalize.update_campaign(
                    campaignArn = campaign['campaignArn'],
                    solutionVersionArn = solution_version_arn,
                    minProvisionedTPS = 1,
                )
                print(json.dumps(response, indent=2))
            except Exception as e:
                print(f'Solution version {solution_name} failed')
                print(e)